# Spark Dataframe Manipulation and EDA

In [1]:
#we use the findspark library to locate spark on our local machine
import findspark
findspark.init('/Applications/spark-2.4.7-bin-hadoop2.7')

In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

import pyspark # only run this after findspark.init()
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import * 
from pyspark.sql.types import * 

### 1. Initialize the Spark Session

We need to begin with initilize the Spark Session. DataFrame can be created and registered as tables. Moreover, SQL tables be executed, tables can be cached, and parquet/json/csv/avro data formatted files can be read.

In [3]:
sc = SparkSession.builder.appName("PysparkExample").config ("spark.sql.shuffle.partitions", "50").config("spark.driver.maxResultSize","5g").config ("spark.sql.execution.arrow.enabled", "true").getOrCreate()

In [4]:
sc

### 2. Load Data

You can download the Kaggle dataset, which includes the book title, author, the date of the best seller list, the published date of the list, the book description, the rank (this week and last week), the publisher, number of weeks on the list, and the price [Link](https://www.kaggle.com/cmenca/new-york-times-hardcover-fiction-best-sellers).

Spark is so Awesome that it supports all different types of data to be read.

DataFrames can be created by reading txt, csv, json and parquet file formats. In our example, we will be using .json formatted file. You can also find and read text, csv and parquet file formats by using the related read functions as shown below.

In [5]:
#JSON
df = sc.read.json('./data/nyt2.json')

#TXT files
# dataframe_txt = sc.read.text('./data/text_data.txt')

#CSV files
# dataframe_csv = sc.read.csv('./data/csv_data.csv')

#PARQUET files
# dataframe_parquet = sc.read.load('./data/parquet_data.parquet')

#### Look at the data with show()

In [6]:
#Show top of dataframe in pandas
df.limit(10).toPandas()

/Applications/spark-2.4.7-bin-hadoop2.7/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: StructType(List(StructField($oid,StringType,true)))
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,_id,amazon_product_url,author,bestsellers_date,description,price,published_date,publisher,rank,rank_last_week,title,weeks_on_list
0,"(5b4aa4ead3089013507db18b,)",http://www.amazon.com/Odd-Hours-Dean-Koontz/dp...,Dean R Koontz,"((1211587200000,),)","Odd Thomas, who can communicate with the dead,...","(None, 27)","((1212883200000,),)",Bantam,"(1,)","(0,)",ODD HOURS,"(1,)"
1,"(5b4aa4ead3089013507db18c,)",http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,"((1211587200000,),)",Aliens have taken control of the minds and bod...,"(25.99, None)","((1212883200000,),)","Little, Brown","(2,)","(1,)",THE HOST,"(3,)"
2,"(5b4aa4ead3089013507db18d,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1211587200000,),)",A woman's happy marriage is shaken when she en...,"(24.95, None)","((1212883200000,),)",St. Martin's,"(3,)","(2,)",LOVE THE ONE YOU'RE WITH,"(2,)"
3,"(5b4aa4ead3089013507db18e,)",http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,"((1211587200000,),)",A Massachusetts state investigator and his tea...,"(22.95, None)","((1212883200000,),)",Putnam,"(4,)","(0,)",THE FRONT,"(1,)"
4,"(5b4aa4ead3089013507db18f,)",http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,"((1211587200000,),)",An aging porn queens aims to cap her career by...,"(24.95, None)","((1212883200000,),)",Doubleday,"(5,)","(0,)",SNUFF,"(1,)"
5,"(5b4aa4ead3089013507db190,)",http://www.amazon.com/Sundays-at-Tiffanys-Jame...,James Patterson and Gabrielle Charbonnet,"((1211587200000,),)",A woman finds an unexpected love,"(24.99, None)","((1212883200000,),)","Little, Brown","(6,)","(3,)",SUNDAYS AT TIFFANY’S,"(4,)"
6,"(5b4aa4ead3089013507db191,)",http://www.amazon.com/Phantom-Prey-John-Sandfo...,John Sandford,"((1211587200000,),)",The Minneapolis detective Lucas Davenport inve...,"(26.95, None)","((1212883200000,),)",Putnam,"(7,)","(4,)",PHANTOM PREY,"(3,)"
7,"(5b4aa4ead3089013507db192,)",http://www.amazon.com/From-Worse-Southern-Vamp...,Jimmy Buffett,"((1211587200000,),)",A Southern family tries to hide its pet pig at...,"(21.99, None)","((1212883200000,),)","Little, Brown","(8,)","(6,)",SWINE NOT?,"(2,)"
8,"(5b4aa4ead3089013507db193,)",http://www.amazon.com/Where-Are-You-Now-Novel/...,Elizabeth George,"((1211587200000,),)","In Cornwall, trying to recover from his wife's...","(27.95, None)","((1212883200000,),)",Harper,"(9,)","(8,)",CARELESS IN RED,"(3,)"
9,"(5b4aa4ead3089013507db194,)",http://www.amazon.com/The-Whole-Truth-David-Ba...,David Baldacci,"((1211587200000,),)",An intelligence agent and a journalist team up...,"(26.99, None)","((1212883200000,),)",Grand Central,"(10,)","(7,)",THE WHOLE TRUTH,"(5,)"


#### Simple Data Inspection 

We usually want to simply skim through the dataframe before going deeper

In [7]:
df.dtypes

[('_id', 'struct<$oid:string>'),
 ('amazon_product_url', 'string'),
 ('author', 'string'),
 ('bestsellers_date', 'struct<$date:struct<$numberLong:string>>'),
 ('description', 'string'),
 ('price', 'struct<$numberDouble:string,$numberInt:string>'),
 ('published_date', 'struct<$date:struct<$numberLong:string>>'),
 ('publisher', 'string'),
 ('rank', 'struct<$numberInt:string>'),
 ('rank_last_week', 'struct<$numberInt:string>'),
 ('title', 'string'),
 ('weeks_on_list', 'struct<$numberInt:string>')]

In [8]:
# Computes summary statistics
df.describe().show()

+-------+--------------------+---------------+--------------------+---------+------------------+
|summary|  amazon_product_url|         author|         description|publisher|             title|
+-------+--------------------+---------------+--------------------+---------+------------------+
|  count|               10195|          10195|               10195|    10195|             10195|
|   mean|                null|           null|                null|     null|1877.7142857142858|
| stddev|                null|           null|                null|     null| 370.9760613506458|
|    min|http://www.amazon...|        AJ Finn|                    |      ACE|  10TH ANNIVERSARY|
|    max|https://www.amazo...|various authors|’Tis for the Rebe...|allantine|               ZOO|
+-------+--------------------+---------------+--------------------+---------+------------------+



In [11]:
# Counts the number of rows in dataframe
df.count()

10195

In [12]:
# Counts the number of distinct rows in dataframe
df.distinct().count()

10195

In [13]:
# Prints plans including physical and logical
df.explain(4)

== Parsed Logical Plan ==
Relation[_id#6,amazon_product_url#7,author#8,bestsellers_date#9,description#10,price#11,published_date#12,publisher#13,rank#14,rank_last_week#15,title#16,weeks_on_list#17] json

== Analyzed Logical Plan ==
_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>
Relation[_id#6,amazon_product_url#7,author#8,bestsellers_date#9,description#10,price#11,published_date#12,publisher#13,rank#14,rank_last_week#15,title#16,weeks_on_list#17] json

== Optimized Logical Plan ==
Relation[_id#6,amazon_product_url#7,author#8,bestsellers_date#9,description#10,price#11,published_date#12,publisher#13,rank#14,rank_last_week#15,t

### 3. Useful common functions

#### [1] Remove Duplicate Values

Duplicate values in a table can be eliminated by using dropDuplicates() function.

In [14]:
#There were no duplicates
df_dropdup = df.dropDuplicates() 
df_dropdup.count()

10195

#### [2] 'Select' Operation 

Select creates a new dataframe using criteria

It is possible to obtain columns by column or by indexing (i.e. dataframe[‘author’]).

In [15]:
#Show all entries in title column
df.select("author").show(10)

#Show all entries in title, author, rank, price columns
df.select("author", "title", "rank", "price").show(10)

+--------------------+
|              author|
+--------------------+
|       Dean R Koontz|
|     Stephenie Meyer|
|        Emily Giffin|
|   Patricia Cornwell|
|     Chuck Palahniuk|
|James Patterson a...|
|       John Sandford|
|       Jimmy Buffett|
|    Elizabeth George|
|      David Baldacci|
+--------------------+
only showing top 10 rows

+--------------------+--------------------+----+--------+
|              author|               title|rank|   price|
+--------------------+--------------------+----+--------+
|       Dean R Koontz|           ODD HOURS| [1]|  [, 27]|
|     Stephenie Meyer|            THE HOST| [2]|[25.99,]|
|        Emily Giffin|LOVE THE ONE YOU'...| [3]|[24.95,]|
|   Patricia Cornwell|           THE FRONT| [4]|[22.95,]|
|     Chuck Palahniuk|               SNUFF| [5]|[24.95,]|
|James Patterson a...|SUNDAYS AT TIFFANY’S| [6]|[24.99,]|
|       John Sandford|        PHANTOM PREY| [7]|[26.95,]|
|       Jimmy Buffett|          SWINE NOT?| [8]|[21.99,]|
|    Elizabeth

In [16]:
df.select('price').show()

+--------+
|   price|
+--------+
|  [, 27]|
|[25.99,]|
|[24.95,]|
|[22.95,]|
|[24.95,]|
|[24.99,]|
|[26.95,]|
|[21.99,]|
|[27.95,]|
|[26.99,]|
|  [, 27]|
|[26.95,]|
|[23.95,]|
|[24.95,]|
|  [, 28]|
|   [, 0]|
|   [, 0]|
|   [, 0]|
|   [, 0]|
|   [, 0]|
+--------+
only showing top 20 rows



In [ ]:
#make dataframe of several columns
cases = cases.select('province','city','infection_case','confirmed')
cases.show()

In [ ]:
# Sort dataframe by highest to lowest value of a column
from pyspark.sql import functions as F

cases.sort(F.desc("confirmed")).show()

#### [3] 'When' Operation 

In [17]:
# Show title and assign 0 or 1 depending on title
df.select("title", when(dataframe.title != 'ODD HOURS', 1).otherwise(0)).show(10)

NameError: name 'dataframe' is not defined

#### [4] 'isin' Operation 

In [18]:
# Show rows with specified authors if in the given options
df[df.author.isin("John Sandford", "Emily Giffin")].toPandas().head()

/Applications/spark-2.4.7-bin-hadoop2.7/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: StructType(List(StructField($oid,StringType,true)))
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


,_id,amazon_product_url,author,bestsellers_date,description,price,published_date,publisher,rank,rank_last_week,title,weeks_on_list
0,"(5b4aa4ead3089013507db18d,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1211587200000,),)",A woman's happy marriage is shaken when she en...,"(24.95, None)","((1212883200000,),)",St. Martin's,"(3,)","(2,)",LOVE THE ONE YOU'RE WITH,"(2,)"
1,"(5b4aa4ead3089013507db191,)",http://www.amazon.com/Phantom-Prey-John-Sandfo...,John Sandford,"((1211587200000,),)",The Minneapolis detective Lucas Davenport inve...,"(26.95, None)","((1212883200000,),)",Putnam,"(7,)","(4,)",PHANTOM PREY,"(3,)"
2,"(5b4aa4ead3089013507db1a2,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1212192000000,),)",A woman’s happy marriage is shaken when she en...,"(24.95, None)","((1213488000000,),)",St. Martin’s,"(4,)","(3,)",LOVE THE ONE YOU'RE WITH,"(3,)"
3,"(5b4aa4ead3089013507db1a7,)",http://www.amazon.com/Phantom-Prey-John-Sandfo...,John Sandford,"((1212192000000,),)",The Minneapolis detective Lucas Davenport inve...,"(26.95, None)","((1213488000000,),)",Putnam,"(9,)","(7,)",PHANTOM PREY,"(4,)"
4,"(5b4aa4ead3089013507db1b6,)",http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,"((1212796800000,),)",A woman’s happy marriage is shaken when she en...,"(24.95, None)","((1214092800000,),)",St. Martin’s,"(4,)","(4,)",LOVE THE ONE YOU'RE WITH,"(4,)"


#### [5] 'Like' Operation 

In [19]:
# Show author and title is TRUE if title has " THE " word in titles
df.select("author", "title", df.title.like("% THE %")).show(15)

+--------------------+--------------------+------------------+
|              author|               title|title LIKE % THE %|
+--------------------+--------------------+------------------+
|       Dean R Koontz|           ODD HOURS|             false|
|     Stephenie Meyer|            THE HOST|             false|
|        Emily Giffin|LOVE THE ONE YOU'...|              true|
|   Patricia Cornwell|           THE FRONT|             false|
|     Chuck Palahniuk|               SNUFF|             false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|             false|
|       John Sandford|        PHANTOM PREY|             false|
|       Jimmy Buffett|          SWINE NOT?|             false|
|    Elizabeth George|     CARELESS IN RED|             false|
|      David Baldacci|     THE WHOLE TRUTH|             false|
|        Troy Denning|          INVINCIBLE|             false|
|          James Frey|BRIGHT SHINY MORNING|             false|
|         Garth Stein|THE ART OF RACING...|            

#### [6] 'Startswith' — 'Endswith' Operation 

StartsWith scans from the beginning of word/content with specified criteria in the brackets. In parallel, EndsWith processes the word/content starting from the end. Both of the functions are case sensitive.

In [20]:
df.select("author", "title", df.title.startswith("THE")).show(5)
df.select("author", "title", df.title.endswith("NT")).show(5)

+-----------------+--------------------+----------------------+
|           author|               title|startswith(title, THE)|
+-----------------+--------------------+----------------------+
|    Dean R Koontz|           ODD HOURS|                 false|
|  Stephenie Meyer|            THE HOST|                  true|
|     Emily Giffin|LOVE THE ONE YOU'...|                 false|
|Patricia Cornwell|           THE FRONT|                  true|
|  Chuck Palahniuk|               SNUFF|                 false|
+-----------------+--------------------+----------------------+
only showing top 5 rows

+-----------------+--------------------+-------------------+
|           author|               title|endswith(title, NT)|
+-----------------+--------------------+-------------------+
|    Dean R Koontz|           ODD HOURS|              false|
|  Stephenie Meyer|            THE HOST|              false|
|     Emily Giffin|LOVE THE ONE YOU'...|              false|
|Patricia Cornwell|           THE

#### [7] 'Substring' Operation 

In the following examples, texts are extracted from the index numbers (1, 3), (3, 6) and (1, 6).

In [22]:
df.select(df.author.substr(1, 3).alias("title")).show(5)
df.select(df.author.substr(3, 6).alias("title")).show(5)
df.select(df.author.substr(1, 6).alias("title")).show(5)

+-----+
|title|
+-----+
|  Dea|
|  Ste|
|  Emi|
|  Pat|
|  Chu|
+-----+
only showing top 5 rows

+------+
| title|
+------+
|an R K|
|epheni|
|ily Gi|
|tricia|
|uck Pa|
+------+
only showing top 5 rows

+------+
| title|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



#### [8] Adding Columns 

In [23]:
from pyspark.sql import functions as F

# Lit() is required while we are creating columns with exact values.
df = df.withColumn('new_column', F.lit('This is a new column'))

display(df)

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

#### [9] Updating Columns 

For updated operations of DataFrame API, withColumnRenamed() function is used with two parameters.

In [ ]:
#Manipulate column by adding 100
import pyspark.sql.functions as F

casesWithNewConfirmed = cases.withColumn("NewConfirmed", 100 + F.col("confirmed"))
casesWithNewConfirmed.show()

In [24]:
# Show title and assign 0 or 1 to new column depending on title
df.select("title", when(df.title != 'ODD HOURS', 1).otherwise(0)).show(10)

+--------------------+-----------------------------------------------------+
|               title|CASE WHEN (NOT (title = ODD HOURS)) THEN 1 ELSE 0 END|
+--------------------+-----------------------------------------------------+
|           ODD HOURS|                                                    0|
|            THE HOST|                                                    1|
|LOVE THE ONE YOU'...|                                                    1|
|           THE FRONT|                                                    1|
|               SNUFF|                                                    1|
|SUNDAYS AT TIFFANY’S|                                                    1|
|        PHANTOM PREY|                                                    1|
|          SWINE NOT?|                                                    1|
|     CARELESS IN RED|                                                    1|
|     THE WHOLE TRUTH|                                                    1|

In [25]:
#Change single column name
cases = df.withColumnRenamed("infection_case","infection_source")

#Change all column name
cases = df.toDF(*['case_id', 'province', 'city', 'group', 'infection_case', 'confirmed',
       'latitude', 'longitude'])

IllegalArgumentException: "requirement failed: The number of columns doesn't match.\nOld column names (13): _id, amazon_product_url, author, bestsellers_date, description, price, published_date, publisher, rank, rank_last_week, title, weeks_on_list, new_column\nNew column names (8): case_id, province, city, group, infection_case, confirmed, latitude, longitude"

In [ ]:
#Change type of column
from pyspark.sql.types import DoubleType, IntegerType, StringType

cases = cases.withColumn('confirmed', F.col('confirmed').cast(IntegerType()))
cases = cases.withColumn('city', F.col('city').cast(StringType()))

cases.show()

In [27]:
# Update column 'amazon_product_url' with 'URL'
df = df.withColumnRenamed('amazon_product_url', 'URL')

df.show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|L

#### [10] Removing Columns 

Removal of a column can be achieved in two ways: \
1. Adding the list of column names in the drop() function 
2. Specifying columns by pointing in the drop function

In [ ]:
#Drop a list of columns
drop_list = ['Dates', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']
data = data.select([column for column in data.columns if column not in drop_list])

In [28]:
# 1.
dataframe_remove = df.drop("publisher", "published_date").show(5)

# 2.
dataframe_remove2 = df.drop(df.publisher).drop(df.published_date).show(5)

+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]| [2]|           [1]|            THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|     Emily Giffin|[[1211587

#### [11] 'GroupBy' Operation

In [29]:
# Group by author, count the books of the authors in the groups

df.groupBy("author").count().show(10)

+--------------------+-----+
|              author|count|
+--------------------+-----+
|          James Frey|    2|
|    Elin Hilderbrand|   58|
|   Sharon Kay Penman|    2|
|         Kate Jacobs|    3|
|       Karen Robards|    6|
|     Gary Shteyngart|    3|
|         Lisa Genova|    7|
|James Patterson a...|   30|
|         Ruth Reichl|    3|
|         JRR Tolkien|    2|
+--------------------+-----+
only showing top 10 rows



In [ ]:
# Top 20 crimes
from pyspark.sql.functions import col
data.groupBy("Category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

#### [12] 'Filter' Operation

In [ ]:
#Filter all the different infection_case in Daegu with more than 10 confirmed cases.
cases.filter((cases.confirmed>10) & (cases.province=='Daegu')).show()

In [32]:
# Filtering entries of title
# Only keeps records having value 'THE HOST'

df.filter(df["title"] == 'THE HOST').show(5)

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|   title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|THE HOST|          [3]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1212192000000]]|Aliens have taken...|[25.99,]|[[1213488000000]]|Little, Brown| [2]|           [2]|THE HOST|          [4]|Th

#### [13] Handling Missing Values

In [33]:
# # Replacing null values
# dataframe.na.fill()
# dataFrame.fillna()
# dataFrameNaFunctions.fill()

# # Returning new dataframe restricting rows with null valuesdataframe.na.drop()
# dataFrame.dropna()
# dataFrameNaFunctions.drop()

# # Return new dataframe replacing one value with another
# dataframe.na.replace(5, 15)
# dataFrame.replace()
# dataFrameNaFunctions.replace()

#### [14] Running SQL Commnads In Spark

In [34]:
# Registering a table
df.registerTempTable("df")

sc.sql("select * from df").show(3)

+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|         author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+---------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|This is a new column|
|[5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Bro

In [ ]:
##Use SQL commands with Spark
cases.registerTempTable('cases_table')
newDF = SQLContext.sql('select * from cases_table where confirmed > 100')
newDF.show()

#### [15] Joining

In [ ]:
# Left Join 'Case' with 'Region' on Province and City column
cases = cases.join(regions, ['province','city'],how='left')
cases.limit(10).toPandas()

#### [16] Functions

In [ ]:
#Spark User Defined Function, confirmed is a column and cases is dataframe
import pyspark.sql.functions as F
from pyspark.sql.types import *

def casesHighLow(confirmed):
    if confirmed < 50: 
        return 'low'
    else:
        return 'high'
    
#convert to a UDF Function by passing in the function and return type of function
casesHighLowUDF = F.udf(casesHighLow, StringType())
CasesWithHighLow = cases.withColumn("HighLow", casesHighLowUDF("confirmed"))
CasesWithHighLow.show()

In [ ]:
def outliers_iqr(data):
    '''
    This function returns the indices of outliers in data.
    
    INPUT:
    data - list containing values
    
    OUTPUT:
    Outlier indices    
    '''
    avg = np.mean(data)
    lower_bound = avg - 2*np.std(data)
    upper_bound = avg + 2*np.std(data)
    return np.where((data > upper_bound) | (data < lower_bound))

#### [17] Write & Save to Files

Any data source type that is loaded to our code as data frames can easily be converted and saved into other types including .parquet and .json

In [ ]:
#Save and load Spark Data at Intermediate Step
#When you work with Spark you will frequently run with memory and storage issues. While in some cases such issues might be resolved using techniques like broadcasting, salting or cache, sometimes just interrupting the workflow and saving and reloading the whole dataframe at a crucial step has helped me a lot. This helps spark to let go of a lot of memory that gets utilized for storing intermediate shuffle data and unused caches.

df.write.parquet("data/df.parquet")
df.unpersist()
spark.read.load("data/df.parquet")

In [ ]:
# Write & Save File in .parquet format
dataframe.select("author", "title", "rank", "description") \
        .write \
        .save("Rankings_Descriptions.parquet")

In [ ]:
# Write & Save File in .json format
dataframe.select("author", "title") \
        .write \
        .save("Authors_Titles.json",format="json")

In [ ]:
#### [17] Write & Save to Files

Any data source type that is loaded to our code as data frames can easily be converted and saved into other types including .parquet and .json

# Write & Save File in Queretarouet format
dataframe.select("author", "title", "rank", "description") \
        .write \
        .save("Rankings_Descriptions.parquet")

# Write & Save File in .json format
dataframe.select("author", "title") \
        .write \
        .save("Authors_Titles.json",format="json")sc.sql("SELECT CASE WHEN description LIKE '%love%' THEN 'Love_Theme'             
           WHEN description LIKE '%hate%' THEN 'Hate_Theme'            
           WHEN description LIKE '%happy%' THEN 'Happiness_Theme' \            
           WHEN description LIKE '%anger%' THEN 'Anger_Theme' \             
           WHEN description LIKE '%horror%' THEN 'Horror_Theme' \              
           WHEN description LIKE '%death%' THEN 'Criminal_Theme' \               
           WHEN description LIKE '%detective%' THEN 'Mystery_Theme' ELSE 'Other_Themes' END Themes from df").groupBy('Themes').count().show()